In [1]:
import pickle 
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [2]:
import sys

In [3]:
sys.path.insert(0, '/content/drive/MyDrive/RecSys/FinalProject/GRU4REC-pytorch-master')

In [ ]:
train_path = '/content/drive/MyDrive/RecSys/FinalProject/data/recSys15TrainOnly.pkl'

In [ ]:
%%time
with open(train_path, 'rb') as fis:
  train_data = pickle.load(fis)

CPU times: user 409 ms, sys: 1.46 s, total: 1.87 s
Wall time: 14.8 s


In [ ]:
df = train_data.df

In [ ]:
%%time
df.sort_values(by=['SessionID', 'Time'], inplace=True)

CPU times: user 36.6 s, sys: 2.57 s, total: 39.1 s
Wall time: 39.2 s


In [ ]:
df.head()

,SessionID,Time,ItemID,item_idx
0,1,1.396857e+09,214536502,0
1770,1,1.396857e+09,214536500,1
2312,1,1.396857e+09,214536506,2
2381,1,1.396857e+09,214577561,3
2520,2,1.396868e+09,214662742,4


In [ ]:
num_embeddings = df['item_idx'].max() + 1
num_embeddings

37483

In [ ]:
get_embeddings = nn.Sequential(
    nn.Embedding(num_embeddings=num_embeddings, embedding_dim=100, padding_idx=num_embeddings-1)
)

model = nn.MultiheadAttention(embed_dim=100, num_heads=1, batch_first=True)

In [ ]:
model = model.to('cuda')

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
)

In [ ]:
%%timeit
groups = df.groupby('SessionID')

for n, g in groups:
  print(n)
  session = g['item_idx']
  session = torch.from_numpy(np.array(session)).view(1, -1)
  emb = get_embeddings(session)
  emb = emb.to('cuda')
  out, _ = model(emb, emb, emb)
  break

1
1
1
1
1
1
1 loop, best of 5: 2.73 s per loop
